In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Dataset

In [4]:
from src.dataset.shape_cor import PairFaustDataset

dataset = PairFaustDataset(
    phase='train',
    data_root='../data/FAUST_r/',
    return_evecs='true',
    return_faces='true',
    num_evecs=200,
    return_corr='true',
    return_dist='true',
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from src.utils.tensor import to_device

data = dataset[1]
data_x, data_y = to_device(data['first'], device), to_device(data['second'], device)
data_x['name'], data_y['name']

('tr_reg_000', 'tr_reg_001')

In [6]:
data_x.keys()

dict_keys(['name', 'verts', 'faces', 'evecs', 'evecs_trans', 'evals', 'mass', 'L', 'dist', 'corr'])

In [7]:
data_x['corr'].shape

torch.Size([5000])

In [33]:
for id in range(100):
    data = dataset[id]
    print(data['first']['verts'].shape, data['first']['corr'].shape)

torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999, 3]) torch.Size([5000])
torch.Size([4999

In [ ]:
import torch
from torch.nn.functional import pad

def pad_collate_fn(batch):
    """
    batch: list of samples, each sample is a dict like 
        {'first': {'field1': tensor or str, ...}, 'second': {'field1': tensor or str, ...}}
    Returns a dict of dicts, with padded tensors where appropriate and lists elsewhere.
    """
    out = {}

    # keys at the first level: e.g. 'first', 'second'
    for part in batch[0].keys():
        out[part] = {}
        fields = batch[0][part].keys()

        for field in fields:
            values = [sample[part][field] for sample in batch]

            if field in ['verts', 'faces', 'evecs']:
                # Tensor field: pad and stack
                lengths = [v.shape[-2] for v in values]
                max_len = max(lengths)
                padded = []

                for v in values:
                    pad_amt = (0, 0) + (0, max_len - v.shape[-2])
                    padded_v = pad(v, pad_amt, "constant", 0)
                    padded.append(padded_v)
                out[part][field] = torch.stack(padded)
                
                # Also provide mask for valid (unpadded) elements
                # out[part][field + '_mask'] = torch.arange(max_len)[None, :] < torch.tensor(lengths)[:, None]

            elif field in ['evals', 'evecs_trans', 'mass', 'corr']:
                # Tensor field: pad and stack
                lengths = [v.shape[-1] for v in values]
                max_len = max(lengths)
                padded = []

                for v in values:
                    pad_amt = (0, max_len - v.shape[-1])
                    padded_v = pad(v, pad_amt, "constant", 0)
                    padded.append(padded_v)
                out[part][field] = torch.stack(padded)
                
                # Also provide mask for valid (unpadded) elements
                # out[part][field + '_mask'] = torch.arange(max_len)[None, :] < torch.tensor(lengths)[:, None]

            elif field in ['L', 'dist']:
                # Tensor field: pad and stack
                lengths = [v.shape[-2] for v in values]
                max_len = max(lengths)
                padded = []

                for v in values:
                    pad_amt = (0, max_len - v.shape[-2]) + (0, max_len - v.shape[-2])
                    padded_v = pad(v, pad_amt, "constant", 0)
                    padded.append(padded_v)
                out[part][field] = torch.stack(padded)
                
                # Also provide mask for valid (unpadded) elements
                # out[part][field + '_mask'] = torch.arange(max_len)[None, :] < torch.tensor(lengths)[:, None]
            
            else:
                # Non-tensor field: collect in list
                out[part][field] = values
    return out

In [18]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=8, collate_fn=pad_collate_fn)

for batch in loader:
    # batch is a dict (if using the collate_fn above)
    single_sample = batch  # keys: 'verts', 'mask', etc.
    break  # Only take the first sample

# batch = to_device(batch, device)

In [24]:
batch['first'].keys()

dict_keys(['name', 'verts', 'faces', 'evecs', 'evecs_trans', 'evals', 'mass', 'L', 'dist', 'corr'])

In [ ]:
batch['first']['corr'].shape

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_shape_batch(batch):
    """
    Custom collate function for shape correspondence dataset.
    Pads all tensors to the maximum number of vertices in the batch.
    """
    # Find maximum number of vertices in the batch
    max_verts_first = max([sample['first']['verts'].shape[0] for sample in batch])
    max_verts_second = max([sample['second']['verts'].shape[0] for sample in batch])
    
    # Find maximum number of faces (if you want to batch faces too)
    max_faces_first = max([sample['first']['faces'].shape[0] for sample in batch])
    max_faces_second = max([sample['second']['faces'].shape[0] for sample in batch])
    
    collated_batch = []
    
    for sample in batch:
        collated_sample = {'first': {}, 'second': {}}
        
        for shape_key in ['first', 'second']:
            shape_data = sample[shape_key]
            max_verts = max_verts_first if shape_key == 'first' else max_verts_second
            max_faces = max_faces_first if shape_key == 'first' else max_faces_second
            current_verts = shape_data['verts'].shape[0]
            current_faces = shape_data['faces'].shape[0]
            
            # Copy non-tensor data
            collated_sample[shape_key]['name'] = shape_data['name']
            
            # Pad vertex coordinates (V, 3) -> (max_V, 3)
            verts_padded = torch.zeros(max_verts, 3)
            verts_padded[:current_verts] = shape_data['verts']
            collated_sample[shape_key]['verts'] = verts_padded
            
            # Pad faces (F, 3) -> (max_F, 3)
            # Note: padded faces will have invalid indices, you might want to set them to -1
            faces_padded = torch.full((max_faces, 3), -1, dtype=shape_data['faces'].dtype)
            faces_padded[:current_faces] = shape_data['faces']
            collated_sample[shape_key]['faces'] = faces_padded
            
            # Pad eigenvectors (V, K) -> (max_V, K)
            K = shape_data['evecs'].shape[1]
            evecs_padded = torch.zeros(max_verts, K)
            evecs_padded[:current_verts] = shape_data['evecs']
            collated_sample[shape_key]['evecs'] = evecs_padded
            
            # Pad transposed eigenvectors (K, V) -> (K, max_V)
            evecs_trans_padded = torch.zeros(K, max_verts)
            evecs_trans_padded[:, :current_verts] = shape_data['evecs_trans']
            collated_sample[shape_key]['evecs_trans'] = evecs_trans_padded
            
            # Eigenvalues don't need padding (K,)
            collated_sample[shape_key]['evals'] = shape_data['evals']
            
            # Pad Laplacian matrix (V, V) -> (max_V, max_V)
            L_padded = torch.zeros(max_verts, max_verts)
            L_padded[:current_verts, :current_verts] = shape_data['L']
            collated_sample[shape_key]['L'] = L_padded
            
            # Pad distance matrix (V, V) -> (max_V, max_V)
            dist_padded = torch.zeros(max_verts, max_verts)
            dist_padded[:current_verts, :current_verts] = shape_data['dist']
            collated_sample[shape_key]['dist'] = dist_padded
            
            # Pad correspondence (V,) -> (max_V,)
            corr_padded = torch.full((max_verts,), -1, dtype=shape_data['corr'].dtype)
            # print(corr_padded.shape, shape_data['corr'].shape, current_verts)
            corr_padded[:len(shape_data['corr'])] = shape_data['corr']
            collated_sample[shape_key]['corr'] = corr_padded
            
            # Create a mask indicating real vs padded vertices
            mask = torch.zeros(max_verts, dtype=torch.bool)
            mask[:current_verts] = True
            collated_sample[shape_key]['mask'] = mask
            
            # Store original number of vertices and faces for reference
            collated_sample[shape_key]['num_verts'] = current_verts
            collated_sample[shape_key]['num_faces'] = current_faces
        
        collated_batch.append(collated_sample)
    
    # Stack all samples in the batch
    batched_data = {'first': {}, 'second': {}}
    
    for shape_key in ['first', 'second']:
        # Stack all tensor fields
        for field in ['verts', 'faces', 'evecs', 'evecs_trans', 'evals', 'L', 'dist', 'corr', 'mask']:
            batched_data[shape_key][field] = torch.stack([
                sample[shape_key][field] for sample in collated_batch
            ])
        
        # Keep non-tensor fields as lists
        batched_data[shape_key]['name'] = [sample[shape_key]['name'] for sample in collated_batch]
        batched_data[shape_key]['num_verts'] = torch.tensor([sample[shape_key]['num_verts'] for sample in collated_batch])
        batched_data[shape_key]['num_faces'] = torch.tensor([sample[shape_key]['num_faces'] for sample in collated_batch])
    
    return batched_data

# Usage with DataLoader
dataloader = DataLoader(
    dataset, 
    batch_size=8, 
    shuffle=True, 
    # num_workers=2,
    collate_fn=collate_shape_batch
)

# Example of how to use the batched data
for batch in dataloader:
    # batch['first']['verts'] has shape (batch_size, max_verts, 3)
    # batch['first']['mask'] has shape (batch_size, max_verts) - True for real vertices
    
    first_verts = batch['first']['verts']  # (B, max_V, 3)
    first_mask = batch['first']['mask']    # (B, max_V)
    
    # When processing, you can use the mask to ignore padded vertices
    # For example, in loss computation:
    # loss = compute_loss(predictions[first_mask], targets[first_mask])
    
    break

: 

In [48]:
def collate_shape_batch_optimized(batch):
    """
    Optimized version using more efficient tensor operations
    """
    batch_size = len(batch)
    
    # Pre-calculate all dimensions to avoid repeated calculations
    first_verts = [sample['first']['verts'].shape[0] for sample in batch]
    second_verts = [sample['second']['verts'].shape[0] for sample in batch]
    first_faces = [sample['first']['faces'].shape[0] for sample in batch]
    second_faces = [sample['second']['faces'].shape[0] for sample in batch]
    
    max_verts_first = max(first_verts)
    max_verts_second = max(second_verts)
    max_faces_first = max(first_faces)
    max_faces_second = max(second_faces)
    
    # Get K (number of eigenvectors) - should be the same for all samples
    K = batch[0]['first']['evecs'].shape[1]
    
    # Get data types from first sample to ensure consistency
    verts_dtype = batch[0]['first']['verts'].dtype
    faces_dtype = batch[0]['first']['faces'].dtype
    evecs_dtype = batch[0]['first']['evecs'].dtype
    evals_dtype = batch[0]['first']['evals'].dtype
    L_dtype = batch[0]['first']['L'].dtype
    dist_dtype = batch[0]['first']['dist'].dtype
    corr_dtype = batch[0]['first']['corr'].dtype
    
    # Initialize output tensors directly with proper shapes and types
    batched_data = {
        'first': {
            'verts': torch.zeros(batch_size, max_verts_first, 3, dtype=verts_dtype),
            'faces': torch.full((batch_size, max_faces_first, 3), -1, dtype=faces_dtype),
            'evecs': torch.zeros(batch_size, max_verts_first, K, dtype=evecs_dtype),
            'evecs_trans': torch.zeros(batch_size, K, max_verts_first, dtype=evecs_dtype),
            'evals': torch.zeros(batch_size, K, dtype=evals_dtype),
            'L': torch.zeros(batch_size, max_verts_first, max_verts_first, dtype=L_dtype),
            'dist': torch.zeros(batch_size, max_verts_first, max_verts_first, dtype=dist_dtype),
            'corr': torch.full((batch_size, max_verts_first), -1, dtype=corr_dtype),
            'mask': torch.zeros(batch_size, max_verts_first, dtype=torch.bool),
            'num_verts': torch.tensor(first_verts, dtype=torch.long),
            'num_faces': torch.tensor(first_faces, dtype=torch.long),
            'name': []
        },
        'second': {
            'verts': torch.zeros(batch_size, max_verts_second, 3, dtype=verts_dtype),
            'faces': torch.full((batch_size, max_faces_second, 3), -1, dtype=faces_dtype),
            'evecs': torch.zeros(batch_size, max_verts_second, K, dtype=evecs_dtype),
            'evecs_trans': torch.zeros(batch_size, K, max_verts_second, dtype=evecs_dtype),
            'evals': torch.zeros(batch_size, K, dtype=evals_dtype),
            'L': torch.zeros(batch_size, max_verts_second, max_verts_second, dtype=L_dtype),
            'dist': torch.zeros(batch_size, max_verts_second, max_verts_second, dtype=dist_dtype),
            'corr': torch.full((batch_size, max_verts_second), -1, dtype=corr_dtype),
            'mask': torch.zeros(batch_size, max_verts_second, dtype=torch.bool),
            'num_verts': torch.tensor(second_verts, dtype=torch.long),
            'num_faces': torch.tensor(second_faces, dtype=torch.long),
            'name': []
        }
    }
    
    # Fill tensors efficiently
    for i, sample in enumerate(batch):
        # Process 'first' shape
        v1 = first_verts[i]
        f1 = first_faces[i]
        
        # Use tensor slicing for efficient copying
        batched_data['first']['verts'][i, :v1] = sample['first']['verts']
        batched_data['first']['faces'][i, :f1] = sample['first']['faces']
        batched_data['first']['evecs'][i, :v1] = sample['first']['evecs']
        batched_data['first']['evecs_trans'][i, :, :v1] = sample['first']['evecs_trans']
        batched_data['first']['evals'][i] = sample['first']['evals']
        batched_data['first']['L'][i, :v1, :v1] = sample['first']['L']
        batched_data['first']['dist'][i, :v1, :v1] = sample['first']['dist']
        batched_data['first']['corr'][i, :len(sample['first']['corr'])] = sample['first']['corr']
        batched_data['first']['mask'][i, :v1] = True
        batched_data['first']['name'].append(sample['first']['name'])
        
        # Process 'second' shape
        v2 = second_verts[i]
        f2 = second_faces[i]
        
        batched_data['second']['verts'][i, :v2] = sample['second']['verts']
        batched_data['second']['faces'][i, :f2] = sample['second']['faces']
        batched_data['second']['evecs'][i, :v2] = sample['second']['evecs']
        batched_data['second']['evecs_trans'][i, :, :v2] = sample['second']['evecs_trans']
        batched_data['second']['evals'][i] = sample['second']['evals']
        batched_data['second']['L'][i, :v2, :v2] = sample['second']['L']
        batched_data['second']['dist'][i, :v2, :v2] = sample['second']['dist']
        batched_data['second']['corr'][i, :len(sample['second']['corr'])] = sample['second']['corr']
        batched_data['second']['mask'][i, :v2] = True
        batched_data['second']['name'].append(sample['second']['name'])
    
    return batched_data

dataloader = DataLoader(
    dataset, 
    batch_size=8, 
    shuffle=True, 
    # num_workers=8,
    collate_fn=collate_shape_batch_optimized,
)

# Example of how to use the batched data
for batch in dataloader:
    # batch['first']['verts'] has shape (batch_size, max_verts, 3)
    # batch['first']['mask'] has shape (batch_size, max_verts) - True for real vertices
    
    first_verts = batch['first']['verts']  # (B, max_V, 3)
    first_mask = batch['first']['mask']    # (B, max_V)
    
    # When processing, you can use the mask to ignore padded vertices
    # For example, in loss computation:
    # loss = compute_loss(predictions[first_mask], targets[first_mask])
    
    break

In [52]:
def collate_shape_batch_ultra_optimized(batch):
    """
    Ultra-optimized version using vectorized operations where possible
    """
    batch_size = len(batch)
    
    # Extract all data first to minimize attribute access
    first_data = [sample['first'] for sample in batch]
    second_data = [sample['second'] for sample in batch]
    
    def process_shape_batch(shape_data_list, shape_name):
        # Pre-calculate dimensions
        verts_counts = [data['verts'].shape[0] for data in shape_data_list]
        faces_counts = [data['faces'].shape[0] for data in shape_data_list]
        max_verts = max(verts_counts)
        max_faces = max(faces_counts)
        K = shape_data_list[0]['evecs'].shape[1]
        
        # Get dtypes from first sample
        sample_data = shape_data_list[0]
        dtypes = {
            'verts': sample_data['verts'].dtype,
            'faces': sample_data['faces'].dtype,
            'evecs': sample_data['evecs'].dtype,
            'evals': sample_data['evals'].dtype,
            'L': sample_data['L'].dtype,
            'dist': sample_data['dist'].dtype,
            'corr': sample_data['corr'].dtype,
        }
        
        # Initialize output tensors
        result = {
            'verts': torch.zeros(batch_size, max_verts, 3, dtype=dtypes['verts']),
            'faces': torch.full((batch_size, max_faces, 3), -1, dtype=dtypes['faces']),
            'evecs': torch.zeros(batch_size, max_verts, K, dtype=dtypes['evecs']),
            'evecs_trans': torch.zeros(batch_size, K, max_verts, dtype=dtypes['evecs']),
            'evals': torch.zeros(batch_size, K, dtype=dtypes['evals']),
            'L': torch.zeros(batch_size, max_verts, max_verts, dtype=dtypes['L']),
            'dist': torch.zeros(batch_size, max_verts, max_verts, dtype=dtypes['dist']),
            'corr': torch.full((batch_size, max_verts), -1, dtype=dtypes['corr']),
            'mask': torch.zeros(batch_size, max_verts, dtype=torch.bool),
            'num_verts': torch.tensor(verts_counts, dtype=torch.long),
            'num_faces': torch.tensor(faces_counts, dtype=torch.long),
            'name': [data['name'] for data in shape_data_list]
        }
        
        # Vectorized filling where possible
        for i, (data, v_count, f_count) in enumerate(zip(shape_data_list, verts_counts, faces_counts)):
            # Use .copy_() for potentially faster copying
            result['verts'][i, :v_count].copy_(data['verts'])
            result['faces'][i, :f_count].copy_(data['faces'])
            result['evecs'][i, :v_count].copy_(data['evecs'])
            result['evecs_trans'][i, :, :v_count].copy_(data['evecs_trans'])
            result['evals'][i].copy_(data['evals'])
            result['L'][i, :v_count, :v_count].copy_(data['L'])
            result['dist'][i, :v_count, :v_count].copy_(data['dist'])
            result['corr'][i, :len(data['corr'])].copy_(data['corr'])
            result['mask'][i, :v_count] = True
        
        return result
    
    # Process both shapes
    first_result = process_shape_batch(first_data, 'first')
    second_result = process_shape_batch(second_data, 'second')
    
    return {'first': first_result, 'second': second_result}

dataloader = DataLoader(
    dataset, 
    batch_size=8, 
    shuffle=True, 
    # num_workers=2,
    collate_fn=collate_shape_batch_ultra_optimized,
)

# Example of how to use the batched data
for batch in dataloader:
    # batch['first']['verts'] has shape (batch_size, max_verts, 3)
    # batch['first']['mask'] has shape (batch_size, max_verts) - True for real vertices
    
    first_verts = batch['first']['verts']  # (B, max_V, 3)
    first_mask = batch['first']['mask']    # (B, max_V)
    
    # When processing, you can use the mask to ignore padded vertices
    # For example, in loss computation:
    # loss = compute_loss(predictions[first_mask], targets[first_mask])
    
    break

## DiffusionNet

In [15]:
from src.module.diffusionnet import DiffusionNet

feature_extractor = DiffusionNet(
    in_channels=128,
    out_channels=256,
    cache_dir='../data/FAUST_r/diffusion',
    input_type='wks',
).to(device)
feature_extractor.eval()

DiffusionNet(
  (first_linear): Linear(in_features=128, out_features=128, bias=True)
  (last_linear): Linear(in_features=128, out_features=256, bias=True)
  (blocks): ModuleList(
    (0): DiffusionNetBlock(
      (diffusion): LearnedTimeDiffusion()
      (gradient_features): SpatialGradientFeatures(
        (A_re): Linear(in_features=128, out_features=128, bias=False)
        (A_im): Linear(in_features=128, out_features=128, bias=False)
      )
      (mlp): MiniMLP(
        (miniMLP_linear_000): Linear(in_features=384, out_features=128, bias=True)
        (miniMLP_activation_000): ReLU()
        (miniMLP_dropout_001): Dropout(p=0.5, inplace=False)
        (miniMLP_linear_001): Linear(in_features=128, out_features=128, bias=True)
        (miniMLP_activation_001): ReLU()
        (miniMLP_dropout_002): Dropout(p=0.5, inplace=False)
        (miniMLP_linear_002): Linear(in_features=128, out_features=128, bias=True)
      )
    )
    (1): DiffusionNetBlock(
      (diffusion): LearnedTimeDiff

In [25]:
feat_x_batch = feature_extractor(
    batch['first']['verts'],
    batch['first']['faces'],
)

In [10]:
feat_x = feature_extractor(
    data_x['verts'].unsqueeze(0),
    data_x['faces'].unsqueeze(0),
)

In [37]:
import torch.nn.functional as F
verts = data_x['verts']  # e.g. [4999, 3]
faces = data_x['faces']  # e.g. [9994, 3]
verts_pad = F.pad(verts, (0, 0, 0, 1), "constant", 0) # e.g. [5000, 3]
faces_pad = F.pad(faces, (0, 0, 0, 2), "constant", 0) # e.g. [9996, 3]
verts.shape, verts_pad.shape, faces.shape, faces_pad.shape

(torch.Size([4999, 3]),
 torch.Size([5000, 3]),
 torch.Size([9994, 3]),
 torch.Size([9996, 3]))

In [44]:
verts_pad[-1, :]

tensor([0., 0., 0.], device='cuda:0')

In [40]:
feat_x = feature_extractor(
    verts.unsqueeze(0),
    faces.unsqueeze(0),
)
feat_x.shape

torch.Size([1, 4999, 256])

In [41]:
feat_x_pad = feature_extractor(
    verts_pad.unsqueeze(0),
    faces_pad.unsqueeze(0),
)
feat_x_pad.shape

torch.Size([1, 5000, 256])

In [57]:
(feat_x - feat_x_pad[0, :-1, :]).norm()

tensor(0.0007, device='cuda:0', grad_fn=<NormBackward1>)

In [54]:
verts_batch = torch.stack([verts_pad, data_y['verts']], dim=0)
faces_batch = torch.stack([faces_pad, data_y['faces']], dim=0)
verts_batch.shape, faces_batch.shape

(torch.Size([2, 5000, 3]), torch.Size([2, 9996, 3]))

In [55]:
feat_x_batch = feature_extractor(
    verts_batch,
    faces_batch,
)
feat_x_batch.shape

torch.Size([2, 5000, 256])

In [60]:
(feat_x - feat_x_batch[0, :-1, :]).norm()

tensor(0.0012, device='cuda:0', grad_fn=<NormBackward1>)

### Throughput

In [28]:
import time

for size in range(1, 9):
    loader = DataLoader(
        dataset,
        batch_size=size,
        collate_fn=pad_collate_fn,
    )

    for batch, data in enumerate(loader):
        data = to_device(data, device)
        print(data['first']['verts'].shape)
        start_time = time.time()
        for _ in range(10):
            feat_x_batch = feature_extractor(
                data['first']['verts'],
                data['first']['faces'],
            )
        end_time = time.time()
        print(f'size: {size} time: {(end_time - start_time):.4f} throughput: {(size * 10 / (end_time - start_time)):.4f} sample/second')

        break

torch.Size([1, 4999, 3])
size: 1 time: 0.1654 throughput: 60.4560 sample/second
torch.Size([2, 4999, 3])
size: 2 time: 0.3456 throughput: 57.8706 sample/second
torch.Size([3, 4999, 3])
size: 3 time: 0.4803 throughput: 62.4624 sample/second


torch.Size([4, 4999, 3])
size: 4 time: 0.6472 throughput: 61.8069 sample/second
torch.Size([5, 4999, 3])
size: 5 time: 0.7743 throughput: 64.5778 sample/second
torch.Size([6, 4999, 3])
size: 6 time: 0.9187 throughput: 65.3091 sample/second
torch.Size([7, 4999, 3])
size: 7 time: 1.1300 throughput: 61.9484 sample/second
torch.Size([8, 4999, 3])
size: 8 time: 1.2073 throughput: 66.2641 sample/second


In [ ]:
import time

for size in range(9, 16):
    loader = DataLoader(
        dataset,
        batch_size=size,
        collate_fn=pad_collate_fn,
    )

    for batch, data in enumerate(loader):
        data = to_device(data, device)
        print(data['first']['verts'].shape)
        start_time = time.time()
        for _ in range(10):
            feat_x_batch = feature_extractor(
                data['first']['verts'],
                data['first']['faces'],
            )
        end_time = time.time()
        print(f'size: {size} throughput: {(size * 10 / (end_time - start_time)):.4f} sample/second')

        break

size: 9 throughput: 65.7034 sample/second
size: 10 throughput: 65.8434 sample/second
size: 11 throughput: 65.4665 sample/second
size: 12 throughput: 64.6875 sample/second
size: 13 throughput: 66.5755 sample/second
size: 14 throughput: 61.2642 sample/second
size: 15 throughput: 67.1292 sample/second
